In [1]:
import os
import re
import subprocess
from pathlib import Path

IN_TXT = "test.txt"
OUT_DIR = "responses_dump"

def split_into_curl_commands(text: str) -> list[str]:
    """
    Finder alle curl-kommandoer i en tekst som typisk er 'Copy as cURL'.
    Vi splitter på linjer der starter med 'curl ' og tager alt indtil næste 'curl '.
    """
    lines = text.splitlines()
    starts = [i for i, line in enumerate(lines) if line.lstrip().startswith("curl ")]
    if not starts:
        raise ValueError("Fandt ingen linjer der starter med 'curl ' i filen.")

    cmds = []
    for idx, s in enumerate(starts):
        e = starts[idx + 1] if idx + 1 < len(starts) else len(lines)
        block = "\n".join(lines[s:e]).strip()

        # Fjern evt trailing semikolon, så shell ikke brokker sig
        if block.endswith(";"):
            block = block[:-1].rstrip()

        cmds.append(block)
    return cmds

def main():
    txt = Path(IN_TXT).read_text(encoding="utf-8", errors="replace")
    cmds = split_into_curl_commands(txt)

    os.makedirs(OUT_DIR, exist_ok=True)

    seen_urls = set()
    kept_cmds = []

    # Valgfrit: dedupe på URL så du ikke henter samme 20 gange
    url_re = re.compile(r"curl\s+'([^']+)'")
    for c in cmds:
        m = url_re.search(c)
        url = m.group(1) if m else None
        if url and url in seen_urls:
            continue
        if url:
            seen_urls.add(url)
        kept_cmds.append(c)

    print(f"Fandt {len(cmds)} curl commands, kører {len(kept_cmds)} efter dedupe.")

    for i, cmd in enumerate(kept_cmds, start=1):
        out_path = Path(OUT_DIR) / f"response_{i:04d}.json"

        # Kør curl og gem stdout som json
        # shell=True fordi Copy-as-cURL ofte bruger backslashes og quotes
        res = subprocess.run(
            cmd,
            shell=True,
            capture_output=True,
            text=True
        )

        if res.returncode != 0:
            err_path = Path(OUT_DIR) / f"response_{i:04d}.error.txt"
            err_path.write_text(res.stderr or "", encoding="utf-8")
            print(f"[{i}/{len(kept_cmds)}] FEJL. Gemte error til {err_path}")
            continue

        body = (res.stdout or "").strip()
        if not body:
            print(f"[{i}/{len(kept_cmds)}] Tom response.")
            continue

        out_path.write_text(body, encoding="utf-8")
        print(f"[{i}/{len(kept_cmds)}] OK -> {out_path}")

    print("Færdig.")

if __name__ == "__main__":
    main()

Fandt 108 curl commands, kører 108 efter dedupe.
[1/108] OK -> responses_dump/response_0001.json
[2/108] OK -> responses_dump/response_0002.json
[3/108] OK -> responses_dump/response_0003.json
[4/108] OK -> responses_dump/response_0004.json
[5/108] OK -> responses_dump/response_0005.json
[6/108] OK -> responses_dump/response_0006.json
[7/108] OK -> responses_dump/response_0007.json
[8/108] OK -> responses_dump/response_0008.json
[9/108] OK -> responses_dump/response_0009.json
[10/108] OK -> responses_dump/response_0010.json
[11/108] OK -> responses_dump/response_0011.json
[12/108] OK -> responses_dump/response_0012.json
[13/108] OK -> responses_dump/response_0013.json
[14/108] OK -> responses_dump/response_0014.json
[15/108] OK -> responses_dump/response_0015.json
[16/108] OK -> responses_dump/response_0016.json
[17/108] OK -> responses_dump/response_0017.json
[18/108] OK -> responses_dump/response_0018.json
[19/108] OK -> responses_dump/response_0019.json
[20/108] OK -> responses_dump